In [73]:
# import Pkg; Pkg.add("POMDPPolicies")
# Pkg.add("POMDPModelTools")
# Pkg.add("Distributions")
# Pkg.add("Combinatorics")
# Pkg.add("StaticArrays")
# Pkg.add("FileIO")
# Pkg.add("JLD2")
# Pkg.add("TickTock")
# Pkg.add("POMDPTools")
# Pkg.add("D3Trees")
# Pkg.add("Random")
# Pkg.add("DataFrames")
# Pkg.add("CSV")

In [74]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [75]:
global Number_level = 4;
global Action_num = 4;
gamma=0.95

States=Array((1:Number_level))
Observations=Array((1:Number_level))

4-element Vector{Int64}:
 1
 2
 3
 4

In [76]:
Observation_matrix=[
    0.8 0.2 0.0 0.0;
    0.1 0.8 0.1 0.0;
    0.0 0.2 0.8 0.0;
    0.0 0.0 0.0 1.0
]

4×4 Matrix{Float64}:
 0.8  0.2  0.0  0.0
 0.1  0.8  0.1  0.0
 0.0  0.2  0.8  0.0
 0.0  0.0  0.0  1.0

In [77]:
T=zeros(Number_level,Number_level,100);

# Load transition probability matrix for each component
for i in 1:100
    fullname = "C:/Users/hhu98/OneDrive - UW-Madison/POMDP/code/Heterogeneous system/hetero_tm/tm"*string(i);
   
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    # the last term in argument of T, which is written as 'i' here,
    # is the index of component in the heterogeneous system
    T[:,:,i].=Transition_matrix; 
end

In [78]:
NumberUnits=20
limit=12
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

In [79]:
Initialstates = zeros(Number_level)
Initialstates[1]=1.0
Initialstates
Initialstate = SparseCat(States, Initialstates)

Initialobs = SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

## Belief updater

In [80]:
function update_belief(b,s_b,a,o,i) 
    # NOTE: 
    # in input, b is belief from last epoch (before a)
    # a is the last action we took
    # o is the new observation after a
    updated_belief = zeros(Number_level)
    if o==Number_level
        updated_belief[Number_level]=1.0
        return updated_belief
    end
    
    if s_b >= a
    # We assume perfect maintenance and it is know to decision maker(belief updater)
        for sp in 1: Number_level
            #p_obs=pdf(SparseCat(Observations, Observation_matrix[sp,:]),o)
            p_obs=Observation_matrix[sp,o]
            p_trans = T[1,sp,i]
            updated_belief[sp] = p_obs * p_trans
        end
        updated_belief ./= sum(updated_belief)
        return updated_belief
    end
    
    for sp in 1:Number_level
        #p_obs=pdf(SparseCat(Observations, Observation_matrix[sp,:]),o)
        p_obs=Observation_matrix[sp,o]
        p_trans = 0.0
        for s in 1:Number_level
            p_trans += T[s,sp,i] * b[s]
        end
        updated_belief[sp] = p_obs * p_trans
    end
        # Normalize the updated belief state to ensure it sums up to 1
        updated_belief ./= sum(updated_belief)
    return updated_belief
end

update_belief (generic function with 1 method)

## Define POMDP

In [81]:
# s is the system-level state vector
#s,r,o,true_a=approximate(s,s_belief,local_a,234) 
# Approximating the solution by taking the state of highest belief as the true state
function approximate(s,s_belief,a,rng=234)
    r=0.0
    prevent_repair= false
    failed_components=count(x->x==Number_level,s_belief)
    if failed_components >= limit
        r += system_penalty
    end

    for i in 1:NumberUnits
        if s_belief[i] >= a 
            # Maintenance action is decided based on belief, not true state
            # since we don't know true states when making decision
            if s_belief[i] == Number_level
                r += failure_penalty
                if prevent_repair == false
                    r += setup_cost
                    prevent_repair = true
                end
            else
                r += maintenance_penalty
                if prevent_repair == false
                    r += setup_cost
                    prevent_repair = true
                end
            end
            # s[i]=rand(crd[1]) ## Substitute crd with T[]
            s[i] = rand(Categorical(T[1,:,i]))

            r += normal_operation
        else
            # s[i] = rand(crd[s[i]])
            s[i] = rand(Categorical(T[s[i],:,i]))
            r = r+normal_operation
        end
    end

    obs=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
        # obs_distribution = observation_function(s[i])
        obs[i] = rand(obs_distribution)
    end

    return (s=s,r=r,o=obs,a=a)
end

approximate (generic function with 2 methods)

In [82]:
function findnN()
    println("Finding best nN policy...")
    rewards_nN = zeros(Number_level,Number_level);
    rewards_nN = rewards_nN.+(-100000000.0);
    temp_rewards = zeros(repetition,1);
    # global rewards_nN_std = zeros(Number_level,Number_level);
    # global rewards_nN_std = rewards_nN_std.+(-100000000.0);
    for N in 2:(Number_level)
        for n in 1:N
            # temp_rewards = zeros(repetition,1)
            Threads.@threads  for j in 1:repetition
            #for j in 1:repetition
                s=repeat(1:1,NumberUnits)
                # o=s 
                o = rand(Initialobs, NumberUnits)
                # belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
                belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
                R_sequence=[]
                for k in 1:simsteps
                    # s_belief = [argmax(row) for row in eachrow(belief)]
                    s_belief = argmax.(belief)
                    local_a = Number_level +1; # when local_a = 0, no repair unless system fails
                    for i in 1:NumberUnits
                    # if there is any unit reaching or exceeding threshold N, 
                    # maintain all the units reaching or exceeding threshold n.
                        
                        if s_belief[i]>= N
                            local_a=n
                        end
                    end
                    true_a=local_a
                    # s is the system-level state vector
                    s,r,o,true_a=approximate(s,s_belief,local_a,234) 
                    for i in 1: NumberUnits
                        belief[i]=update_belief(belief[i], s_belief[i],true_a, o[i],i)
                    end
                    append!(R_sequence,r)
                end
                R_discounted=0
                for k in 1:simsteps
                    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
                end
                temp_rewards[j]=R_discounted
            end
            rewards_nN[n,N] = mean(temp_rewards)
            #rewards_nN_std[n,N] = std(temp_rewards)
        end
    end
    # This max_reward is actually wrong in POMDP
    # The (n, N) obtained should be calculated based on true state
    (max_reward,nN)=findmax(rewards_nN)
    
    #println("Max rewards of (n, N)-policy is", max_reward, "n is ", nN[1], "N is ", nN[2])
    #return max_reward, rewards_nN_std[nN], nN
    return nN
end

findnN (generic function with 1 method)

In [83]:
function generative(s,s_belief,a,rng=234)
    r=0.0

    if a <= Number_level
         r += setup_cost
    end

    true_failed_components=count(x->x==Number_level,s)
    if true_failed_components >= limit
        r += system_penalty
    end

    for i in 1:NumberUnits
        if s_belief[i] >= a
            if s[i] == Number_level
                r += failure_penalty
            else
                r += maintenance_penalty
            end
            s[i] = rand(Categorical(T[1,:,i]))
            r += normal_operation
        else
            s[i] = rand(Categorical(T[s[i],:,i]))
            r = r+normal_operation
        end
    end

    obs=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        obs_distribution = SparseCat(Observations, Observation_matrix[s[i],:])
        # obs_distribution = observation_function(s[i])
        obs[i] = rand(obs_distribution)
    end

    return (s=s,r=r,o=obs)
end

generative (generic function with 2 methods)

In [84]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],mean=[],std=[],nN=[])
cost=[ 
    [0	-800	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 


for units in [20, 25, 30, 40, 50, 60]

    for K in 2:2:convert(Int64, floor((units+1)/2)) 
        
        global NumberUnits=convert(Int64,units)
        global limit=convert(Int64,K)
        # global crd = Array{Categorical}(undef,Number_level)
        # for i in 1:Number_level
        #     global crd[i] = Categorical(Transition_matrix[i,:]);
        # end
            
        # global initialb= zeros(NumberUnits, Number_level)  
        # global initialb[:, 1] .= 1.0
        for c in cost
            tick()
            println(units)  
            print("K : ")
            println(K)
            global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
            println(c)  
            global NumberUnits=units

            global simsteps = 100;
            global repetition = 10000;
            global rewards_nN = zeros(Number_level,Number_level);
            global rewards_nN = rewards_nN.+(-100000000.0);
            global rewards_nN_std = zeros(Number_level,Number_level);
            global rewards_nN_std = rewards_nN_std.+(-100000000.0);

            global discount_factor =gamma
            global temp_rewards = zeros(repetition,1);
            # Looking for optimal (n, N)
            nN =findnN()
            n=nN[1]
            N=nN[2]
            
            trials=10000;
            results=zeros(trials)*0.1
            # cnt_maintenance = zeros(trials)
            rewards=0.0
            rewards_std=0.0
            Threads.@threads for h in 1:trials
                s=repeat(1:1,NumberUnits)
                # o=s 
                o = rand(Initialobs, NumberUnits)
                # belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
                belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
                R_sequence=[]
              
                
                # maintenance_count = 0
                for k in 1:simsteps
                    s_belief = argmax.(belief)
                    local_a = Number_level +1; 
                    for i in 1:NumberUnits
                    # If there is any unit reaching or exceeding threshold N, 
                    # maintain all the units reaching or exceeding threshold n,
                    # otherwise, no maintenance.
                    # As a result, when system fails, there are components at failure level,
                    # so N must be reached.
                        if s_belief[i]>= N
                            local_a=n
                        end
                    end
                    s,r,o=generative(s,s_belief,local_a,234) 
                    for i in 1: NumberUnits
                        # modified: belief[i,:] to belief[i]
                        belief[i]=update_belief(belief[i], s_belief[i],local_a, o[i],i)
                    end
                    append!(R_sequence,r)
                end
                R_discounted=0.0
                for k in 1:simsteps
                    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
                end
                # cnt_maintenance[h]=maintenance_count
                results[h]=R_discounted
            end
            rewards=mean(results)
            rewards_std=std(results)
            # N_maintenance = mean(cnt_maintenance)
            println("Rewards of (n, N)-policy is", rewards, "n is ", nN[1], "N is ", nN[2])
            # println("Number of maintenance times is ", N_maintenance)
            pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,rewards,rewards_std,nN])
        end
    end
end

20
K : 2
[0 -800 -100 -1200 -2000]
Finding best nN policy...


[ Info:  started timer at: 2025-03-08T16:24:39.048


Rewards of (n, N)-policy is-20568.72618558247n is 2N is 3
20
K : 2
[0 -1200 -100 -1200 -2000]
Finding best nN policy...


[ Info:  started timer at: 2025-03-08T16:27:55.460


Rewards of (n, N)-policy is-25089.074923473425n is 2N is 3
20
K : 2
[0 -1600 -100 -1200 -2000]
Finding best nN policy...


...

Rewards of (n, N)-policy is-59879.03215076481n is 2N is 4
60
K : 30
[0 -2800 -100 -1200 -2000]
Finding best nN policy...


[ Info:  started timer at: 2025-03-10T12:32:10.038


Rewards of (n, N)-policy is-62968.265890700626n is 2N is 4
60
K : 30
[0 -3200 -100 -1200 -2000]
Finding best nN policy...


[ Info:  started timer at: 2025-03-10T12:41:30.275


Rewards of (n, N)-policy is-66054.62431604795n is 2N is 4


In [85]:
df

Row,u,K,n,s,m,f,p,mean,std,nN
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,60,30,0,-3200,-100,-1200,-2000,-66054.6,3258.32,"CartesianIndex(2, 4)"
2,60,30,0,-2800,-100,-1200,-2000,-62968.3,3134.34,"CartesianIndex(2, 4)"
3,60,30,0,-2400,-100,-1200,-2000,-59879.0,2990.57,"CartesianIndex(2, 4)"
4,60,30,0,-2000,-100,-1200,-2000,-56798.6,2908.56,"CartesianIndex(2, 4)"
5,60,30,0,-1600,-100,-1200,-2000,-53703.8,2808.01,"CartesianIndex(2, 4)"
6,60,30,0,-1200,-100,-1200,-2000,-50680.9,2677.21,"CartesianIndex(2, 4)"
7,60,30,0,-800,-100,-1200,-2000,-47514.1,2592.19,"CartesianIndex(2, 4)"
8,60,28,0,-3200,-100,-1200,-2000,-66061.6,3340.46,"CartesianIndex(2, 4)"
9,60,28,0,-2800,-100,-1200,-2000,-62966.1,3169.47,"CartesianIndex(2, 4)"


In [86]:
using CSV
CSV.write("Hetero (n,N)-policy.csv", df)

"Hetero (n,N)-policy.csv"